In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/My Drive/Github

/content/gdrive/My Drive/Github


In [4]:
%cd FYP-FP-Generator-Model

/content/gdrive/My Drive/Github/FYP-FP-Generator-Model


# Step 1: Set Up Your Environment on Google Colab
Connect to a GPU runtime

Go to the top menu: Runtime > Change Runtime Type > Hardware Accelerator > GPU

In [5]:
!pip install opencv-python pillow matplotlib

# Step 2: Upload Your Floor Plan Image
# Step 3: Load the Image

In [6]:
import os
import cv2
import numpy as np
import json
from datetime import datetime
import matplotlib.pyplot as plt

In [7]:
# Base directories
RAW_DATA_DIR = "./data/raw/"
PREPROCESSED_DIR = "./data/preprocessed/"

In [8]:
%cd ./data/

/content/gdrive/My Drive/Github/FYP-FP-Generator-Model/data


Step 4: Normalize the Image Size

Step 5: Normalize Pixel Values

Step 6: Extract Room Colors

1.   Define the color codes:

In [9]:
# RGB color coding for rooms
color_map = {
    "Bedroom": (255, 0, 0),                # Red
    "Bathroom": (0, 0, 255),               # Blue
    "Kitchen": (255, 165, 0),              # Orange
    "Drawing Room": (0, 128, 0),           # Green
    "Garage": (165, 42, 42),               # Brown
    "Lounge": (255, 255, 0),               # Yellow
    "Backyard": (50, 205, 50),             # Lime Green
    "Stairs": (0, 128, 128),               # Teal
    "Store": (128, 0, 128),                # Purple
    "Open Space": (0, 255, 255),           # Cyan
    "Prayer Room": (127, 127, 127),        # Crimson
    "Staircase": (153, 51, 255),           # RGB(153, 51, 255)
    "Lobby": (255, 0, 255),                # Magenta
    "Lawn": (64, 224, 208),                # Turquoise
    "Dining": (255, 192, 203),             # Pink
    "Servant Quarters": (75, 0, 130),      # Indigo
    "Passage": (128, 128, 0),              # Olive Green
    "Laundry": (230, 230, 250),            # Lavender
    "Dressing Area": (255, 127, 80),       # Coral
    "Side Garden": (255, 215, 0),          # Gold
    "Library": (255, 191, 0),              # Amber
    "Wall": (0, 0, 0),                     # Black
    "Door": (128, 0, 0),                   # Mahogany
}


2.   Extract each room based on its color:

In [10]:
def display_image(image, title="Image", figsize=(6, 6), is_bgr=True):
    """
    Displays an image using Matplotlib.

    Args:
        image (numpy.ndarray): The image to display.
        title (str): Title for the plot.
        figsize (tuple): Figure size for the plot.
        is_bgr (bool): If True, converts the image from BGR to RGB for visualization.
    """
    if is_bgr:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=figsize)
    plt.imshow(image)
    plt.axis("off")
    plt.title(title)
    plt.show()

def process_floor_plan(fp_path, plot_size):
    """
    Processes a single floor plan, extracts metadata, and saves the output.
    """
    file_name = os.path.basename(fp_path)
    file_base, file_ext = os.path.splitext(file_name)  # Extract base filename without extension

    # Extract metadata from file name
    file_parts = file_base.split("_")
    if len(file_parts) < 5:
        print(f"Skipping invalid file name: {file_name}")
        return

    _, floor_level, plan_type, fp_number, version = file_parts[:5]

    # Load the image
    img = cv2.imread(fp_path)
    if img is None:
        print(f"Error loading image: {fp_path}")
        return

    # Display Original Image
    display_image(img, title="Original Floor Plan")

    # Resize image while maintaining aspect ratio
    target_width, target_height = 512, 928
    resized_img = cv2.resize(img, (target_width, target_height))

    # Normalize pixel values
    normalized_img = resized_img / 255.0

    # Display resized image
    display_image(resized_img, title=f"Resized Floor Plan ({target_width}x{target_height})")

    # Detect rooms and create masks
    detected_rooms = {}
    for room, color in color_map.items():
        bgr_color = color[::-1]  # Convert RGB to BGR for OpenCV
        mask = cv2.inRange(resized_img, np.array(bgr_color) - 10, np.array(bgr_color) + 10)

        if cv2.countNonZero(mask) > 0:
            detected_rooms[room] = mask
            display_image(mask, title=f"{room} Mask", is_bgr=False)

    # Generate metadata
    metadata = {
        "PlotSize": plot_size,
        "FloorLevel": floor_level,
        "PlanType": plan_type,
        "FP_Number": fp_number,
        "Version": version,
        "ImageDimensions": {"Width": target_width, "Height": target_height},
        "DPI": 96,
        "AspectRatio": round(target_width / target_height, 3),
        "ScalingFactor": 0.0045,
        "ProcessingDate": datetime.utcnow().isoformat(),
        "DetectedRooms": list(detected_rooms.keys()),
        "RoomDetails": []
    }

    # Calculate room areas
    for room, mask in detected_rooms.items():
        area_px = cv2.countNonZero(mask)
        area_sqft = area_px * metadata["ScalingFactor"]
        metadata["RoomDetails"].append({
            "RoomType": room,
            "AreaInPixels": area_px,
            "ApproxAreaInSqFt": round(area_sqft, 2)
        })

    # Ensure correct folder naming
    fp_folder = file_base
    fp_dir = os.path.join(PREPROCESSED_DIR, plot_size, fp_folder)
    os.makedirs(fp_dir, exist_ok=True)

    # Save preprocessed image
    preprocessed_image_path = os.path.join(fp_dir, f"{file_base}.png")
    cv2.imwrite(preprocessed_image_path, (normalized_img * 255).astype("uint8"))

    # Save masks
    masks_dir = os.path.join(fp_dir, "masks")
    os.makedirs(masks_dir, exist_ok=True)
    for room, mask in detected_rooms.items():
        mask_filename = os.path.join(masks_dir, f"{file_base}_{room}_mask.png")
        cv2.imwrite(mask_filename, mask)

    # Save metadata
    metadata_path = os.path.join(fp_dir, f"{file_base}.json")
    with open(metadata_path, "w") as f:
        json.dump(metadata, f, indent=4)

    print(f"✅ Processed and saved: {fp_folder}")

Step 7: Calculate Room Areas
1. Define the total image size (e.g., 5 Marla is 25 x 45 feet):

In [ ]:
# Define the single image details
single_fp_path = f"{RAW_DATA_DIR}/5_marla/raster/5Marla_GF_FP_001_V01.png"  # Update with actual path
single_plot_size = "5_marla"

# Run the processing function on this single image
process_floor_plan(single_fp_path, single_plot_size)